In [6]:
# 初始化环境，可以设置成全局的环境变量，也可以在代码中指定
from openai import OpenAI
import os

# 我自建的openai代理
# os.environ['OPENAI_BASE_URL'] = 'https://proxy.hehanwang.com/v1'
# 填写你自己的key
# os.environ['OPENAI_API_KEY'] = 'sk-xxx'

client = OpenAI()

# 1.上传文件
file = client.files.create(
    file=open('data/提示词框架：BROKE.md', 'rb'),
    purpose='assistants')
print(file.model_dump_json())

# 2.创建助手
assistant = client.beta.assistants.create(
    instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    name="RAG Assistant",
    file_ids=[file.id]
)
print(assistant.model_dump_json())

{"id":"file-kS0Zhq21HPm9HNGWYBaGCeIY","bytes":6520,"created_at":1704096208,"filename":"提示词框架：BROKE.md","object":"file","purpose":"assistants","status":"processed","status_details":null}
{"id":"asst_Cn3Wue2SGENhBOXCsyDavmje","created_at":1704096209,"description":null,"file_ids":["file-kS0Zhq21HPm9HNGWYBaGCeIY"],"instructions":"You are a customer support chatbot. Use your knowledge base to best respond to customer queries.","metadata":{},"model":"gpt-4-1106-preview","name":"RAG Assistant","object":"assistant","tools":[{"type":"retrieval"}]}


In [7]:
import time

# 3.创建对话
run = client.beta.threads.create_and_run(
    assistant_id=assistant.id,
    thread={
        "messages": [
            {"role": "user", "content": "什么是BROK？"}
        ]
    }
)
print(run.model_dump_json())

# 4.获取对话结果
run = client.beta.threads.runs.retrieve(
    thread_id=run.thread_id,
    run_id=run.id
)
print(run.model_dump_json())

while run.status not in {'completed', 'failed', 'expired', 'cancelled'}:
    run = client.beta.threads.runs.retrieve(
        thread_id=run.thread_id,
        run_id=run.id
    )
    print(run.model_dump_json())
    time.sleep(10)

thread_messages = client.beta.threads.messages.list(thread_id=run.thread_id)
print(thread_messages.model_dump_json())

{"id":"run_AIHpFK4DTj31uuvYyG2sPr5c","assistant_id":"asst_Cn3Wue2SGENhBOXCsyDavmje","cancelled_at":null,"completed_at":null,"created_at":1704097096,"expires_at":1704097696,"failed_at":null,"file_ids":["file-kS0Zhq21HPm9HNGWYBaGCeIY"],"instructions":"You are a customer support chatbot. Use your knowledge base to best respond to customer queries.","last_error":null,"metadata":{},"model":"gpt-4-1106-preview","object":"thread.run","required_action":null,"started_at":null,"status":"queued","thread_id":"thread_zu60CQF55WIOF2tOquwcsiAg","tools":[{"type":"retrieval"}]}
{"id":"run_AIHpFK4DTj31uuvYyG2sPr5c","assistant_id":"asst_Cn3Wue2SGENhBOXCsyDavmje","cancelled_at":null,"completed_at":null,"created_at":1704097096,"expires_at":1704097696,"failed_at":null,"file_ids":["file-kS0Zhq21HPm9HNGWYBaGCeIY"],"instructions":"You are a customer support chatbot. Use your knowledge base to best respond to customer queries.","last_error":null,"metadata":{},"model":"gpt-4-1106-preview","object":"thread.run","